In [1]:
signuature = ''

## Daily Aggregrage
- dictionary is done
- aggreage both domain, tag and tag_gp

### 0. Initialization

In [1]:
import cx_Oracle
import pandas as pd
from smtutil import common
from smtutil import hiveutil
from datetime import datetime
import numpy as np
import csv, pickle

Connect to Oracle Successfully
Connect to Oracle Successfully


In [2]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [3]:
hive_db_name = 'weblog_v2'

In [4]:
sc, spark = hiveutil.get_sc_spark()
hiveutil.register_common_udf(spark)

In [11]:
# from datetime import datetime
# import pyspark.sql.functions as F
# # print(datetime.now())
# # sdf = spark.read.parquet('/app/ws-ic/hive/weblog.db/t_domain_daily/').filter(F.col('date_id')==20230116)
# print(datetime.now())
# sdf = spark.read.parquet('/app/ws-ic/hive/weblog.db/t_domain_daily/date_id=20230116')
# sdf.limit(1).show()
# print(datetime.now())
# sdf2 = spark.read.parquet('/app/ws-ic/hive/weblog_v2.db/t_domain_daily/date_id=20230116')
# sdf3 = spark.read.parquet('/app/ws-ic/hive/weblog.db/t_domain_index/')
# sdf4 = spark.read.parquet('/app/ws-ic/hive/weblog_v2.db/t_domain_index/')
# print(sdf3.count(), sdf4.count())

In [5]:
spark.sql("""
refresh PRD_BIZ_SUMM_VW.VW_IDP_SSL_SED_CALL_SUMM
""")

DataFrame[]

In [6]:
dates_to_loaded = spark.sql(f"""
with t_loaded as (
    select distinct
        date_id
    from {hive_db_name}.t_domain_daily
), t_legal as (
    select distinct
        date_id
    from PRD_BIZ_SUMM_VW.VW_IDP_SSL_SED_CALL_SUMM
)
select 
    t_legal.date_id
from t_legal
left join t_loaded
    on t_legal.date_id = t_loaded.date_id
where t_loaded.date_id is null
and t_legal.date_id >= 20200101
order by t_legal.date_id desc
limit 4
""").toPandas().date_id
dates_to_loaded = list(dates_to_loaded)
dates_to_loaded

AnalysisException: "Table or view not found: `weblog_v2`.`t_domain_daily`; line 5 pos 9;\n'GlobalLimit 4\n+- 'LocalLimit 4\n   +- 'Sort ['t_legal.date_id DESC NULLS LAST], true\n      +- 'Project ['t_legal.date_id]\n         +- 'Filter (isnull('t_loaded.date_id) && ('t_legal.date_id >= 20200101))\n            +- 'Join LeftOuter, ('t_legal.date_id = 't_loaded.date_id)\n               :- SubqueryAlias `t_legal`\n               :  +- Distinct\n               :     +- Project [date_id#5]\n               :        +- SubqueryAlias `prd_biz_summ_vw`.`vw_idp_ssl_sed_call_summ`\n               :           +- Relation[subr_num#0L,ssl_cert_domain#1,byte_count#2L,hour_binary#3,total_hit#4L,date_id#5] parquet\n               +- 'SubqueryAlias `t_loaded`\n                  +- 'Distinct\n                     +- 'Project ['date_id]\n                        +- 'UnresolvedRelation `weblog_v2`.`t_domain_daily`\n"

### 1. Aggregation

In [8]:
spark.sql(f"refresh {hive_db_name}.t_domain_daily")
spark.sql(f"""
with t as (
    select distinct
        date_id
    from {hive_db_name}.t_domain_daily
)
select
    left(date_id, 6) as month_id,
    count(*) as days
from t
group by month_id
order by month_id
""").toPandas()

AnalysisException: "Table or view not found: `weblog_v2`.`t_domain_daily`; line 5 pos 9;\n'Sort ['month_id ASC NULLS FIRST], true\n+- 'Aggregate ['month_id], ['left('date_id, 6) AS month_id#6, count(1) AS days#7L]\n   +- 'SubqueryAlias `t`\n      +- 'Distinct\n         +- 'Project ['date_id]\n            +- 'UnresolvedRelation `weblog_v2`.`t_domain_daily`\n"

In [8]:
spark.sql(f"refresh {hive_db_name}.t_tag_daily")
spark.sql(f"""
with t as (
    select distinct
        date_id
    from {hive_db_name}.t_tag_daily
)
select
    left(date_id, 6) as month_id,
    count(*) as days
from t
group by month_id
order by month_id
""").toPandas()

month_id  days
0    202001    31
1    202002    29
2    202003    31
3    202004    30
4    202005    31
5    202006    30
6    202007    31
7    202008    31
8    202009    30
9    202010    31
10   202011    30
11   202012    31
12   202101    31
13   202102    28
14   202103    31
15   202104    30
16   202105    31
17   202106    28
18   202107    31
19   202108    31
20   202109    30
21   202110    31
22   202111    30
23   202112    31
24   202201    15
25   202208     3
26   202209     2
27   202210     3

In [9]:
spark.sql(f"refresh {hive_db_name}.t_tag_gp_daily")
spark.sql(f"""
with t as (
    select distinct
        date_id
    from {hive_db_name}.t_tag_gp_daily
)
select
    left(date_id, 6) as month_id,
    count(*) as days
from t
group by month_id
order by month_id
""").toPandas()

month_id  days
0    202001    31
1    202002    29
2    202003    31
3    202004    30
4    202005    31
5    202006    30
6    202007    31
7    202008    31
8    202009    30
9    202010    31
10   202011    30
11   202012    31
12   202101    31
13   202102    28
14   202103    31
15   202104    30
16   202105    31
17   202106    28
18   202107    31
19   202108    31
20   202109    30
21   202110    31
22   202111    30
23   202112    31
24   202201    14
25   202208     3
26   202209     2
27   202210     3

#### 1.1. Aggregate Domain

In [7]:
dates_to_loaded=[20230116]

In [8]:
for date_id in dates_to_loaded: # 20 mins
    print(date_id, datetime.now())
    #spark.sql(f"alter table {hive_db_name}.t_domain_daily drop if exists partition (date_id={date_id})")
    spark.sql(f"""
    select
        subr_num,
        domain_index as agg_value,
        'domain' as agg_level,
        0 as in_byte,
        0 as out_byte,
        byte_count,
        total_hit,
        0 as duration,
        date_id
    from prd_biz_summ_vw.vw_idp_ssl_sed_call_summ
    join {hive_db_name}.t_domain_index
        on vw_idp_ssl_sed_call_summ.ssl_cert_domain = t_domain_index.domain
    where date_id = {date_id}
    """.format(
        date_id=date_id
    ))\
        .coalesce(10)\
        .write.mode('append')\
        .partitionBy('date_id')\
        .saveAsTable(f'{hive_db_name}.t_domain_daily')
print(datetime.now())

20230116 2023-01-17 12:00:57.110675
2023-01-17 12:02:44.974205


In [12]:
# %%javascript
# alert('done')

### 1.2. Aggregate Tag

In [13]:
spark.sql(f"""
select *
from {hive_db_name}.t_tag_dictionary
limit 2
""").toPandas()

Empty DataFrame
Columns: [tag_gp, tag, domain, agg_level, agg_value, default_aggregation]
Index: []

In [14]:
for date_id in dates_to_loaded: # 20 mins        
    print(date_id, datetime.now()) # 1 min
    spark.sql(f"alter table {hive_db_name}.t_tag_daily drop if exists partition (date_id={date_id})")
    spark.sql(f"""
    with t_tag_dictionary as (
        select
            t_tag_dictionary.tag_gp,
            t_tag_dictionary.tag,

            t_tag_dictionary.agg_value,
            t_tag_dictionary.agg_level,

            t_domain_dictionary.agg_value as domain_index
        from {hive_db_name}.t_tag_dictionary
        join {hive_db_name}.t_domain_dictionary
            on t_tag_dictionary.tag_gp = t_domain_dictionary.tag_gp
            and t_tag_dictionary.tag = t_domain_dictionary.tag
        where t_tag_dictionary.default_aggregation = 1
    )
    select
        subr_num,
        t_tag_dictionary.agg_value,
        t_tag_dictionary.agg_level,
        sum(in_byte) as in_byte,
        sum(out_byte) as out_byte,
        sum(byte_count) as byte_count,
        sum(total_hit) as total_hit,
        0 as duration,
        date_id
    from {hive_db_name}.t_domain_daily
    join t_tag_dictionary
        on t_tag_dictionary.domain_index = t_domain_daily.agg_value
    where date_id = '{date_id}'
    group by subr_num, date_id, t_tag_dictionary.agg_value, t_tag_dictionary.agg_level
    """.format(
        date_id=date_id
    ))\
        .coalesce(10)\
        .write.mode('append')\
        .partitionBy('date_id')\
        .saveAsTable(f'{hive_db_name}.t_tag_daily')
print(datetime.now())

20230109 2023-01-10 15:43:44.546536
20220828 2023-01-10 15:44:08.253674
20220827 2023-01-10 15:44:13.213706
20220826 2023-01-10 15:44:18.076015
2023-01-10 15:44:22.984748


In [15]:
# %%javascript
# alert('done')

### 1.3. Aggregate Tag Group

In [9]:
for date_id in dates_to_loaded: # 15 mins    
    print(date_id, datetime.now())
    
    spark.sql(f"alter table {hive_db_name}.t_tag_gp_daily drop if exists partition (date_id={date_id})")
    spark.sql(f"""
    with t_tag_dictionary as (
        select
            t_tag_gp_dictionary.tag_gp,
            t_tag_gp_dictionary.tag,

            t_tag_gp_dictionary.agg_value,
            t_tag_gp_dictionary.agg_level,

            t_domain_dictionary.agg_value as domain_index
        from {hive_db_name}.t_tag_gp_dictionary
        join {hive_db_name}.t_domain_dictionary
            on t_tag_gp_dictionary.tag_gp = t_domain_dictionary.tag_gp
        where t_tag_gp_dictionary.default_aggregation = 1
    )
    select
        subr_num,
        t_tag_dictionary.agg_value,
        t_tag_dictionary.agg_level,
        sum(in_byte) as in_byte,
        sum(out_byte) as out_byte,
        sum(byte_count) as byte_count,
        sum(total_hit) as total_hit,
        0 as duration,
        date_id
    from {hive_db_name}.t_domain_daily
    join t_tag_dictionary
        on t_tag_dictionary.domain_index = t_domain_daily.agg_value
    where date_id = '{date_id}'
    group by subr_num, date_id, t_tag_dictionary.agg_value, t_tag_dictionary.agg_level
    """.format(
        date_id=date_id
    ))\
        .coalesce(10)\
        .write.mode('append')\
        .partitionBy('date_id')\
        .saveAsTable(f'{hive_db_name}.t_tag_gp_daily')
print(datetime.now())

20230116 2023-01-17 11:13:49.900255


AnalysisException: "Table or view not found: `weblog_v2`.`t_domain_daily`; line 26 pos 9;\n'Aggregate ['subr_num, 'date_id, 't_tag_dictionary.agg_value, 't_tag_dictionary.agg_level], ['subr_num, 't_tag_dictionary.agg_value, 't_tag_dictionary.agg_level, 'sum('in_byte) AS in_byte#6, 'sum('out_byte) AS out_byte#7, 'sum('byte_count) AS byte_count#8, 'sum('total_hit) AS total_hit#9, 0 AS duration#10, 'date_id]\n+- 'Filter ('date_id = 20230116)\n   +- 'Join Inner, ('t_tag_dictionary.domain_index = 't_domain_daily.agg_value)\n      :- 'UnresolvedRelation `weblog_v2`.`t_domain_daily`\n      +- 'SubqueryAlias `t_tag_dictionary`\n         +- 'Project ['t_tag_gp_dictionary.tag_gp, 't_tag_gp_dictionary.tag, 't_tag_gp_dictionary.agg_value, 't_tag_gp_dictionary.agg_level, 't_domain_dictionary.agg_value AS domain_index#11]\n            +- 'Filter ('t_tag_gp_dictionary.default_aggregation = 1)\n               +- 'Join Inner, ('t_tag_gp_dictionary.tag_gp = 't_domain_dictionary.tag_gp)\n                

In [17]:
# %%javascript
# alert('done')

In [6]:
spark.stop()
sc.stop()